In [1]:
!pip install py2neo
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 3.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 173 kB 4.1 MB/s 
  Created wheel for neo4j: filename=neo4j-5.1.0-py3-none-any.whl size=248097 sha256=a861af992dd27a9c39802bc8aa5d5aa73628e1a53c6834d26b7d61454bdc84d0
  Stored in directory: /root/.cache/pip/wheels/64/96/da/c30ec8577a970eadecc436103fbcbf7af1d6398d0ebe32cbab
Successfully built neo4j


In [2]:
import requests
import json
import csv
import io
from py2neo import Graph
from neo4j import GraphDatabase, basic_auth

In [3]:
def SubmitPMIDList(Pmid,Format,Bioconcept):

    # json = {"Pmids": Pmid}

    json = {}

    #
    # load pmids
    #
    with io.open(Inputfile,'r',encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")
    
    print(json)
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format , json = json)
    
    res = r.json()
    print(res)

    pmid = res['id']
    res0 = res['passages'][0]['annotations']
    res1 = res['passages'][1]['annotations']
    
    result = {}
    result = res0 + res1
    
    for i in result:
        i['Pmid'] = Pmid
    return result
    
    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        return result

In [8]:
Inputfile = "./pmid"
Format = "biocjson"
Bioconcept = ""
res_json=[]

In [9]:
with io.open(Inputfile,'r',encoding="utf-8") as file_input:
    pmidlist = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
for i in pmidlist['pmids']:
    res_json.extend(SubmitPMIDList([i], Format, Bioconcept))

{'pmids': ['35900868']}
{'_id': '35900868|None', 'id': '35900868', 'infons': {}, 'passages': [{'infons': {'journal': 'J Clin Invest;2022Jul28. doi:10.1172/JCI161908', 'year': '2022', 'type': 'title', 'authors': 'Lone MA, Aaltonen MJ, Zidell A, Pedro HF, Morales Saute JA, Mathew S, Mohassel P, Bonnemann CG, Shoubridge EA, Hornemann T, ', 'section': 'Title'}, 'offset': 0, 'text': 'SPTLC1 variants associated with ALS produce distinct sphingolipid signatures through impaired interaction with ORMDL proteins.', 'sentences': [], 'annotations': [{'id': '2', 'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'}, 'text': 'SPTLC1', 'locations': [{'offset': 0, 'length': 6}]}, {'id': '3', 'infons': {'identifier': 'MESH:D013107', 'type': 'Chemical'}, 'text': 'sphingolipid', 'locations': [{'offset': 53, 'length': 12}]}], 'relations': []}, {'infons': {'type': 'abstract', 'section': 'Abstract'}, 'offset': 127, 'text': 'Amyotrophic lateral sclerosis (ALS) is a progressive neurodeg

In [33]:
res_json

[{'id': '2',
  'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'},
  'text': 'SPTLC1',
  'locations': [{'offset': 0, 'length': 6}],
  'Pmid': ['35900868']},
 {'id': '3',
  'infons': {'identifier': 'MESH:D013107', 'type': 'Chemical'},
  'text': 'sphingolipid',
  'locations': [{'offset': 53, 'length': 12}],
  'Pmid': ['35900868']},
 {'id': '31',
  'infons': {'identifier': 'MESH:D000690', 'type': 'Disease'},
  'text': 'Amyotrophic lateral sclerosis',
  'locations': [{'offset': 127, 'length': 29}],
  'Pmid': ['35900868']},
 {'id': '32',
  'infons': {'identifier': 'MESH:D019636', 'type': 'Disease'},
  'text': 'neurodegenerative disease',
  'locations': [{'offset': 180, 'length': 25}],
  'Pmid': ['35900868']},
 {'id': '33',
  'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'},
  'text': 'SPTLC1',
  'locations': [{'offset': 248, 'length': 6}],
  'Pmid': ['35900868']},
 {'id': '34',
  'infons': {'identifier': '189', 'type': 'Gene', 'ncbi_homo

In [59]:
#print([res_json['cars'][i]['model'] for i in range(len(x['cars']))])
f = open('Pubtator_entities.txt', mode='w')

[f.write(res_json[i]['text'] + '\n') for i in range(len(res_json)) if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical')]
f.close()
f = open('Pubtator_entities.txt', mode='r')
print(f.read())
'''
for i in range(len(res_json)):
  if(res_json[i]['infons']['type'] == 'Gene' or res_json[i]['infons']['type'] == 'Chemical'):
    print(res_json[i]['text'], '\n')
'''


SPTLC1
sphingolipid
SPTLC1
serine-palmitoyltransferase
SPT
sphingolipids
SPTLC1
SPT
SPTLC1
sphingolipid
lipid
SPTLC1
HSAN1
SPT
L-alanine
serine
serine
SPTLC1
HSAN1
SPTLC1
HSAN1
1-deoxySL
serine
SPTLC1

SPTLC1 

sphingolipid 

SPTLC1 

serine-palmitoyltransferase 

SPT 

sphingolipids 

SPTLC1 

SPT 

SPTLC1 

sphingolipid 

lipid 

SPTLC1 

HSAN1 

SPT 

L-alanine 

serine 

serine 

SPTLC1 

HSAN1 

SPTLC1 

HSAN1 

1-deoxySL 

serine 

SPTLC1 

